<a href="https://colab.research.google.com/github/tkasod2/TIL/blob/main/prj_analysis/%EC%8B%A4%EC%A0%84_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_10_%EC%9D%B4%ED%83%88_%EC%98%88%EC%B8%A1_%EC%84%B1%EB%8A%A5_%EB%81%8C%EC%96%B4%EC%98%AC%EB%A6%AC%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
DRIVE_PATH = "/content/drive/MyDrive/"
DATA_DIR_PATH = "[바로-써먹는-데이터-분석]-강의자료-new4/2부/Part 5. 우리 서비스의 유저들은 언제-왜 떠나요_ - 고객 이탈 분석/실전 프로젝트 10. 이탈 예측 성능 끌어올리기/"


# 데이터 살펴보기

In [3]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split

In [4]:
# 데이터 살펴보기
# 유저ID - 이탈 여부

train = pd.read_csv(Path(DRIVE_PATH, DATA_DIR_PATH, "train_v2_sample.csv"))
train.sample(5)

,msno,is_churn
65817,E+B/eXX0FekbcxF9ditxOZR2xp+fakkjMadTqHj08WM=,0
52097,HojR7Gn+lQnM75oQnQAY35vI1TMVVYoS0ALSmPpuJng=,1
14809,L7j0qVdrtZLLaE3P2+IVR8unm4NsBvTKvCUQaHtBO2w=,0
20318,DIdJWmBr/PJCYrevpmjSqY2ryt4dFxzmBeNCN7jEfSE=,0
55325,tJiHyV1WNkvzJW4yxmTs0tGW4C9/E5NrpxFZwX0rDds=,0


In [5]:
# 유저ID - 유저 정보
members = pd.read_csv(Path(DRIVE_PATH, DATA_DIR_PATH, "members_v3_sample.csv"), dtype={'gender':'category', 'registered_via': 'category', 'city': 'category'})
members.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,1uvo4CIMRCYGJLiZu029XfvwBqQBiUsMA0ul7+e6kiY=,14,0,NaN,4,20170219
1,20LnYllCVoaG2npEbo9XGABR9r9vTy3PSKxaFkeN6ZQ=,1,0,NaN,7,20100121
2,MvCU75rkklULzbjNSmBcwd0+b2//5javJXXnjdbDL3g=,1,0,NaN,7,20160311
3,h+T+zv9sziRXNmCLSLRBGCu4mPM68/buke6nd+J3FEo=,1,0,NaN,7,20160304
4,Hb0OT/zrvdJE7lYSE5AuPkQ6a/AcJTRxha74GtYNbOA=,17,52,male,9,20090310


In [6]:
# 유저ID - 거래 내역
# 샘플된 데이터
transactions = pd.read_csv(Path(DRIVE_PATH, DATA_DIR_PATH, "transactions_v2.csv"), 
                           dtype = {'payment_method_id' : 'category'})
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,20170329,20170331,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,20170323,20170423,0


In [7]:
# train 데이터와 members 데이터를 합치기
train_members = pd.merge(left = train,right = members, how = 'left',on='msno')
train_members.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time
0,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1,1,0,NaN,7,20140129
1,WkF/FvlzpBLFoa+Hm3hagOfme2pHLE2y+RJ3eGcLT0k=,1,1,0,NaN,7,20140515
2,w2O0vchmncaBEQLoC5Pn7qRoSPiJ6X2x5O+woVFDhnY=,1,13,0,NaN,9,20140605
3,SlkPQrh6oIqsdpW+Sl2adYlJNEwWSQsIYycM11dSUQg=,1,14,49,male,7,20140611
4,Z9Y6f2hXZpPIvbLgHHgUncs7Tb+qmKgGNSwwVAiMlm0=,1,12,21,female,9,20140719


In [8]:
# transaction 데이터에서 정보를 뽑아낼 준비
merged_transactions = pd.merge(left = train_members, right = transactions, how='left', on='msno')
merged_transactions.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1,1,0,NaN,7,20140129,41,30.0,99.0,99.0,1.0,20170307.0,20170307.0,1.0
1,WkF/FvlzpBLFoa+Hm3hagOfme2pHLE2y+RJ3eGcLT0k=,1,1,0,NaN,7,20140515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,w2O0vchmncaBEQLoC5Pn7qRoSPiJ6X2x5O+woVFDhnY=,1,13,0,NaN,9,20140605,40,30.0,149.0,149.0,1.0,20170311.0,20170314.0,1.0
3,SlkPQrh6oIqsdpW+Sl2adYlJNEwWSQsIYycM11dSUQg=,1,14,49,male,7,20140611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Z9Y6f2hXZpPIvbLgHHgUncs7Tb+qmKgGNSwwVAiMlm0=,1,12,21,female,9,20140719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
len(merged_transactions)

105174

In [12]:
# 유저 별 거래 통계량 구하기
# msno : value의 의미가 없으므로 count처리
stats = merged_transactions.groupby('msno').agg(
        {'msno' : 'count', 'plan_list_price': 'sum', 'actual_amount_paid': 'mean'}
    ).rename(
        columns={'msno': 'total_order', 'plan_list_price': 'plan_net_worth', 'actual_amount_paid': 'mean_payment_each_transaction'}
        ).reset_index()

In [13]:
stats.head()

,msno,total_order,plan_net_worth,mean_payment_each_transaction
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,1,99.0,99.0
1,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,1,149.0,149.0
2,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,1,149.0,149.0
3,++BW1PJYU5OZi3n3+IDLiU+d1IL1VE/GLx6p64TDs6U=,1,149.0,149.0
4,++Ck01c3EF07Ejek2jfXlKut+sEfg+0ry+A5uWeL9vY=,1,99.0,99.0


In [14]:
# 유저 정보에 거래 이력 통계 정보를 합치기
df = pd.merge(left=train_members, right=stats, how='left', on='msno')
df.head(10)

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,total_order,plan_net_worth,mean_payment_each_transaction
0,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1,1,0,NaN,7,20140129,1,99.0,99.0
1,WkF/FvlzpBLFoa+Hm3hagOfme2pHLE2y+RJ3eGcLT0k=,1,1,0,NaN,7,20140515,1,0.0,NaN
2,w2O0vchmncaBEQLoC5Pn7qRoSPiJ6X2x5O+woVFDhnY=,1,13,0,NaN,9,20140605,1,149.0,149.0
3,SlkPQrh6oIqsdpW+Sl2adYlJNEwWSQsIYycM11dSUQg=,1,14,49,male,7,20140611,1,0.0,NaN
4,Z9Y6f2hXZpPIvbLgHHgUncs7Tb+qmKgGNSwwVAiMlm0=,1,12,21,female,9,20140719,1,0.0,NaN
5,wLtPucL8EyK8yUp+F9J1S8Ad/D9CRA8kSB0maalwzjE=,1,4,31,female,3,20140828,1,149.0,149.0
6,VCZm6WJqkwDhmConNXKrPly5XJWE26zjE7FL9tlSqww=,1,15,19,male,3,20141112,2,2400.0,1200.0
7,6OlZbbgM+0PjnShnk82ORhLuBaqn6m0bv+I/x9bpmHs=,1,1,34,female,3,20150427,1,0.0,NaN
8,k7bE7mrFGS+PxQfb98pG0HJ8/5OXED8BaH0YCzJWMiA=,1,22,21,female,9,20150806,1,0.0,NaN
9,bDAeje3aBAc11bqBQUTBuZ1gac+kjt24459hhYHD19I=,1,1,0,NaN,9,20150818,1,0.0,NaN


In [15]:
# 아웃라이어 영역 찾고,
df['bd'].min(), df['bd'].max()

(-3152, 1035)

In [28]:
# 더블체크
print('0.999 qunatile : {}'.format(df['bd'].quantile(0.999)))
print('0.001 qunatile : {}'.format(df['bd'].quantile(0.001)))

0.999 qunatile : 72.81100000001607
0.001 qunatile : 0.0


In [29]:
# 아웃라이어 제거
df['bd'].clip(0,100, inplace=True) 
df['bd'].fillna(0,inplace=True)

In [30]:
# 확인
df['bd'].min(), df['bd'].max()

(0, 100)

In [31]:
# 필요없는 컬럼, 결측값 제거
df.drop(columns='msno', inplace=True)
df.dropna(inplace=True)

In [32]:
# 더미화
df_dummies = pd.get_dummies(df)

In [33]:
df_dummies.head()

,is_churn,bd,registration_init_time,total_order,plan_net_worth,mean_payment_each_transaction,city_1,city_10,city_11,city_12,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22,city_3,city_4,city_5,city_6,city_7,city_8,city_9,gender_female,gender_male,registered_via_13,registered_via_3,registered_via_4,registered_via_7,registered_via_9
5,1,31,20140828,1,149.0,149.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
6,1,19,20141112,2,2400.0,1200.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
11,1,22,20150920,1,149.0,149.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
12,1,33,20150922,1,536.0,536.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
14,1,19,20160204,3,447.0,149.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0


In [34]:
X = df_dummies.drop(columns='is_churn')
y = df_dummies['is_churn'].values

In [35]:
# 학습 / 테스트 데이터
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=0)

# 트리 앙상블 모델로 이탈 예측하기

In [36]:
from sklearn.tree import DecisionTreeClassifier

In [37]:
# 모델 초기화 및 학습하기
tree_model = DecisionTreeClassifier(random_state=0, max_depth=2)
tree_model.fit(train_X, train_y)

DecisionTreeClassifier(max_depth=2, random_state=0)

In [38]:
# 모델 평가하기 (정확도)
from sklearn import metrics
prediction_test = tree_model.predict(test_X)
print(metrics.f1_score(test_y, prediction_test))

0.6349717969379532


### 랜덤 포레스트

In [39]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
# 모델 초기화 및 학습하기
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(train_X, train_y)

RandomForestClassifier()

In [49]:
# 모델 평가하기 (정확도)
from sklearn import metrics
prediction_RF = rf_model.predict(test_X)
print(metrics.f1_score(test_y, prediction_RF))

0.6602373887240356


In [50]:
print(metrics.classification_report(test_y, prediction_RF))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      8288
           1       0.85      0.54      0.66       825

    accuracy                           0.95      9113
   macro avg       0.90      0.76      0.82      9113
weighted avg       0.95      0.95      0.94      9113



### 부스팅(Boosting)

Ada boost, Gradient boost, XG Boost

In [ ]:
from sklearn import metrics

In [61]:
%%time
from sklearn.ensemble import AdaBoostClassifier
ADA_clf = AdaBoostClassifier(n_estimators=100)
ADA_clf.fit(train_X, train_y)
prediction_ADA = ada_model.predict(test_X)
print(metrics.f1_score(test_y, prediction_ADA))
print(metrics.classification_report(test_y, prediction_ADA))

0.6518987341772151
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      8288
           1       0.94      0.50      0.65       825

    accuracy                           0.95      9113
   macro avg       0.95      0.75      0.81      9113
weighted avg       0.95      0.95      0.94      9113

CPU times: user 2.45 s, sys: 16.4 ms, total: 2.47 s
Wall time: 2.47 s


In [60]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
GDB_clf = GradientBoostingClassifier(n_estimators = 100)
GDB_clf.fit(train_X, train_y)

prediction_GDB = GDB_clf.predict(test_X)
print(metrics.f1_score(test_y, prediction_GDB))
print(metrics.classification_report(test_y, prediction_GDB))

0.6743827160493826
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      8288
           1       0.93      0.53      0.67       825

    accuracy                           0.95      9113
   macro avg       0.94      0.76      0.82      9113
weighted avg       0.95      0.95      0.95      9113

CPU times: user 3.16 s, sys: 8.43 ms, total: 3.17 s
Wall time: 3.17 s


In [63]:
%%time
from xgboost import XGBClassifier
XGB_clf = XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=1000, verbosity=1, silent=None)
XGB_clf.fit(train_X, train_y)

prediction_XGB = XGB_clf.predict(test_X)
print(metrics.f1_score(test_y, prediction_XGB))
print(metrics.classification_report(test_y, prediction_XGB))

0.676946800308404
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      8288
           1       0.93      0.53      0.68       825

    accuracy                           0.95      9113
   macro avg       0.94      0.76      0.83      9113
weighted avg       0.95      0.95      0.95      9113

CPU times: user 22.4 s, sys: 31.9 ms, total: 22.5 s
Wall time: 22.4 s
